# Generators
***
A generator in python is a function that returns a set of items one at a time. In other words, you can iterate over the functions and retrieve each object.

Generators look exactly like functions but instead of returning elements, you `yield` elements.

In [2]:
def range_gen(n=3):
    i = 0
    while i < n:
        yield i
        i += 1

print(type(range_gen()))

<class 'generator'>


In [3]:
range_5 = range_gen(5)
for i in range_5:
    print(i, end=' ')
    
print()
# or just
for i in range_gen(5):
    print(i, end=' ')

0 1 2 3 4 
0 1 2 3 4 

Above we defined our first generator, `range_gen`. All it does, is feed us the numbers 0 to n, one at a time. 

Every time we call range_5 in the for loop, the generator **yields** the next value. This tells us that it maintains some sort of state remembering where it is. But why do we care? Why not just return the entire list of numbers from 0 to n and iterate over that in the for loop?

The reason is that generators are very memory efficient. Instead of creating and returning a potentially massive list of numbers (which can take up a lot of memory), the generator only has to store and give 1 number at a time.

In [4]:
import sys

In [5]:
def range_list(n):
    return list(range(n))

We'll compare the amount of bytes returned from the `range_list` function vs. the `range_gen` generator.

In [6]:
print(f'{sys.getsizeof(range_list(1_000_000)):,} bytes')
print(f'{sys.getsizeof(range_gen(1_000_000)):,} bytes')

9,000,120 bytes
128 bytes


We can see that the function takes up wayyyyy more memory than the generator does.

We don't just have to iterate over generators in a for loop. We can also call the `next` method on it to retrieve each value. But if we call `next` more times than there are values to yield than python will raise a StopIteration exception.

When we iterate over a generator using a for-loop, in the background python is really just calling `next` until it hits a StopIteration. 

In [7]:
range_5 = range_gen(5)

print(next(range_5), end=' ')
print(next(range_5), end=' ')
print(next(range_5), end=' ')
print(next(range_5), end=' ')
print(next(range_5), end=' ')
# will raise a StopIteration exception
print(next(range_5))

0 1 2 3 4 

StopIteration: 

Before we really move on we'll add some print statements to the generator to really see whats going on.

In [8]:
def range_gen(n=3):
    print('Starting')
    i = 0
    while i < n:
        print('Yielding ', end=' ')
        yield i
        print('Moving on to next value')
        i += 1
    print('Generator done')

In [9]:
for i in range_gen():
    print(i)
    print('======')

Starting
Yielding  0
Moving on to next value
Yielding  1
Moving on to next value
Yielding  2
Moving on to next value
Generator done


So we can see only the first time the generator is called is the start code run. Then once we yield a value the generator pauses and gives control back to the main code (in this case the for loop with print(i)). Then the next time we call the generator the function resumes right where it was and works until it hits the next yield.

In [10]:
# We can also use generators to resemble infinite lists.
def infinity():
    i = 0
    while(True):
        yield i
        i += 1

In [11]:
for i in infinity():
    print(i)
    
    if i > 50:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


In [12]:
infty = infinity()

In [30]:
# run this cell as many times as you want 
next(infty)

17

Once we have a generator if we want the full sequence at once we can just convert it into a list using `list(gen)`. (If you try this with `infty` it won't terminate.)

In [34]:
def range_gen(n=3):
    i = 0
    while i < n:
        yield i
        i += 1

In [37]:
x = list(range_gen(10))
print(type(x))
x

<class 'list'>


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Generator expressions

Just like in lists we have list comprehensions to create a list in one line, we also have generator expressions. 

generator expressions